In [1]:
import pandas as pd
import csv

In [2]:
# file_path = './dataset/small_test.csv'
file_path = './dataset/train.csv'
escape_char_defined = "\\"
# data = pd.read_csv(file_path, error_bad_lines=False)
data = pd.read_csv(file_path, error_bad_lines=False, quoting=csv.QUOTE_NONE, escapechar=escape_char_defined)

/home/abhishek/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


In [3]:
data = data.dropna(axis=0)

In [4]:
data

,TITLE,DESCRIPTION,BULLET_POINTS,BRAND,BROWSE_NODE_ID
0,"Pete The Cat Bedtime Blues Doll, 14.5 Inch","Pete the Cat is the coolest, most popular cat ...","[Pete the Cat Bedtime Blues plush doll,Based o...",MerryMakers,0
1,"The New Yorker NYHM014 Refrigerator Magnet, 2 ...",The New Yorker Handsome Cello Wrapped Hard Mag...,[Cat In A Tea Cup by New Yorker cover artist G...,The New Yorker,1
5,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,Men'S Full Sleeve Raglan T-Shirts Denim T-Shir...,"[Color: Blue,Sleeve: Full Sleeve,Material: Cot...",Bhavya Enterprise,5
6,Glance Women's Wallet (Black) (LW-21),This Black wallet by Glance will be a treasure...,[The Most Comfortable Women's Wallet That You ...,Glance,6
7,Wild Animals Hungry Brain Educational Flash Ca...,Wild Animals are the animals that mostly stays...,[Playful learning: Flash cards develops the lo...,hungry brain,7
...,...,...,...,...,...
2903018,The NXT LVL Liquid Silicone Soft Back Cover Ca...,Product color may slightly vary due to photogr...,[Liquid silicone case compatible with apple pr...,Generic,794481
2903019,Premium Aviator Sunglasses - HD Polarized (Bri...,These premium Aviator Sunglasses with 5 color ...,"[Frame size: Lens height - 56mm, Lens width - ...",Generic,1040
2903020,Social Distance Stickers - Set of 5 Sticker Sl...,set of 5 prints social distancing sticker self...,[covid19 safety sticker - set of 5 to maintain...,Generic,15199
2903021,Torr-to Face Shield PACK OF 5 with Adjustable ...,* COMPLETE FACE PROTECTION: Torr-to Face Shiel...,"[350 MICRONS PACK OF 5 PCS,COMPLETE FACE PROTE...",TORR-TO,1044933


In [6]:
df = data.head(10000)
df.to_csv("./small_train.csv")

In [6]:
import numpy as np
import spacy

# Need to load the large model to get the vectors
nlp = spacy.load('en_core_web_lg')

2021-07-31 15:24:18.362841: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-31 15:24:18.362875: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
X = data[["DESCRIPTION"]].head(100)
# X = data[["TITLE","DESCRIPTION","BULLET_POINTS","BRAND"]].head(100)
y = data[["BROWSE_NODE_ID"]].head(100)

In [21]:
with nlp.disable_pipes():
    desc_vectors = np.array([nlp(data.DESCRIPTION).vector for idx, data in data.iterrows()])
    
desc_vectors.shape

(1000, 300)

In [22]:
vectors = []
vectors.append(desc_vectors)


In [23]:
vectors

[array([[ 0.08789352,  0.09976257, -0.1452722 , ...,  0.02595549,
         -0.03153082, -0.01424141],
        [ 0.09742955,  0.23453149, -0.07009125, ..., -0.07307971,
          0.04925448, -0.00891512],
        [ 0.09488766,  0.26450905, -0.20750545, ...,  0.02458139,
          0.26968375, -0.06735981],
        ...,
        [-0.02991651,  0.17411137, -0.16331376, ..., -0.05811862,
          0.01818301,  0.08772899],
        [-0.00525228,  0.10396843, -0.0484056 , ..., -0.07300182,
         -0.04228529,  0.06796657],
        [-0.08230537,  0.09772008, -0.07884906, ..., -0.19207115,
          0.03850461,  0.12434939]], dtype=float32)]

In [24]:
with nlp.disable_pipes():
    title_vectors = np.array([nlp(data.TITLE).vector for idx, data in data.iterrows()])
    
title_vectors.shape

(1000, 300)

In [25]:
vectors.append(title_vectors)

In [26]:
with nlp.disable_pipes():
    brand_vectors = np.array([nlp(data.BRAND).vector for idx, data in data.iterrows()])
    
brand_vectors.shape

(1000, 300)

In [27]:
vectors.append(brand_vectors)

In [28]:
with nlp.disable_pipes():
    bp_vectors = np.array([nlp(data.BULLET_POINTS).vector for idx, data in data.iterrows()])
    
bp_vectors.shape

(1000, 300)

In [29]:
vectors.append(bp_vectors)

In [ ]:
A = (title_vectors, desc_vectors, brand_vectors, bp_vectors)
R = np.hstack(A)

In [39]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(R, data.BROWSE_NODE_ID, 
                                                    test_size=0.1, random_state=1)

# Create the LinearSVC model
model = LinearSVC(random_state=1, dual=False)
# Fit the model
model.fit(X_train, y_train)

# Uncomment and run to see model accuracy
print(f'Model test accuracy: {model.score(X_test, y_test)*100:.3f}%')

Model test accuracy: 7.000%


In [38]:
R.shape

(1000, 1200)

In [15]:
a=[[1,2,3],[3,4,5]]
b=[[1,2,3],[3,4,5]]
c=[[1,2,3],[3,4,5]]
d=[[1,2,3],[3,4,5]]


In [16]:
e = [a,b,c,d]

In [17]:
e

[[[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]]]

In [18]:
f=[[1,2,3],[23,23,32]]

In [19]:
e.append(f)

In [20]:
e

[[[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [3, 4, 5]],
 [[1, 2, 3], [23, 23, 32]]]

In [8]:
import tensorflow as tf
#Considering y variable holds numpy array
# y_tensor = tf.convert_to_tensor(y, dtype=tf.int64) 

def get_vector(text) :
    filtered_text = ""
    doc = nlp(text)
    for token in doc:
        if(token.is_stop==False):
            filtered_text = filtered_text+" "+str(token)
    doc = nlp(filtered_text)
    vector = doc.vector
    return vector
#     df = pd.DataFrame(vector)
#     return df
#     return np.array(df)
#     a = np.array(vector)
#     return tf.convert_to_tensor(vector)
#     return tf.convert_to_tensor(vector, dtype=tf.float32)

def to_vec(df) :
    df = df.applymap(get_vector)
    return df

In [9]:
from multiprocessing import Pool

def parallelize_dataframe(df, func):
    df_split = np.array_split(df, 4)
    pool = Pool(4)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [10]:
# X=X.applymap(get_vector)
X = parallelize_dataframe(X, to_vec)

In [11]:
X.head(1)["DESCRIPTION"][0].shape

(300,)

In [12]:
X.head(100)

,DESCRIPTION
0,"[0.115266226, 0.06962519, -0.15789133, -0.2083..."
1,"[0.10960721, 0.3174163, -0.058482353, 0.000402..."
2,"[0.11014653, 0.28188354, -0.24703766, 0.057204..."
3,"[-0.1407077, 0.059835803, -0.078086816, -0.150..."
4,"[-0.22405812, 0.031534642, -0.1976032, -0.1937..."
...,...
95,"[-0.09715153, 0.063072465, -0.1360343, 0.00885..."
96,"[0.18526618, 0.20716426, 0.15742008, -0.074156..."
97,"[-0.074659616, 0.27811608, -0.056639906, -0.05..."
98,"[0.09538429, 0.05223043, -0.055029, -0.1767271..."


In [13]:
df = X.head()

In [14]:
df.assign(Area=lambda df: 1*300)
df['Volume'] = 1*300

pd.qcut(df.DESCRIPTION, 5, labels=False)

/tmp/ipykernel_29204/3728845111.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Volume'] = 1*300


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [15]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.3, random_state = 42)

In [16]:
from sklearn.svm import LinearSVC
svc = LinearSVC(random_state=1, dual=False, max_iter=10000)


In [17]:
svc.fit(train_X, train_y)

ValueError: setting an array element with a sequence.

In [ ]:
svc.score(val_X, val_y)

In [ ]:
from keras.models import Sequential
from keras import layers

input_dim = train_X.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', 
               optimizer='adam', 
               metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(train_X, train_y,
                     epochs=100,
                     verbose=False,
                     validation_data=(val_X, val_y),
                     batch_size=10)

In [ ]:
>>> from keras.backend import clear_session
>>> clear_session()

In [ ]:
>>> loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
>>> print("Training Accuracy: {:.4f}".format(accuracy))
>>> loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
>>> print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
#create model
model = Sequential()
#add model layers
model.add(Conv2D(64, kernel_size=3, input_shape=(28,28,1)))
model.add(Conv2D(32, kernel_size=3))
model.add(Flatten())
model.add(Dense(10))

In [ ]:
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
#train the model
model.fit(train_X, train_y, validation_data=(val_X, val_y), epochs=3)

In [ ]:
#predict first 4 images in the test set
model.predict(val_X)

In [ ]:
desc_vec_df = get_vector_df("DESCRIPTION")

data["DESCRIPTION_VECTOR"] = desc_vec_df
desc_vec_df

In [ ]:
title_vec_df = get_vector_df("TITLE")
title_vec_df
data["TITLE_VECTOR"] = title_vec_df

In [ ]:
bp_vec_df = get_vector_df("BULLET_POINTS")
bp_vec_df
data["BP_VECTOR"] = bp_vec_df

In [ ]:
# X :
X = data[["DESCRIPTION_VECTOR","TITLE_VECTOR","BP_VECTOR"]]
X

In [ ]:
# Y :
y = data[["BROWSE_NODE_ID"]]
y

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.svm import LinearSVC
# # X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.1, random_state=1)
# # Set dual=False to speed up training, and it's not needed
# svc = LinearSVC(random_state=1, dual=False, max_iter=10000)
# svc.fit(X_train, y_train)
# print(f"Accuracy: {svc.score(X_test, y_test) * 100:.3f}%", )

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
# split data into training and validation data, for both features and target
# The split is based on a random number generator. Supplying a numeric value to
# the random_state argument guarantees we get the same split every time we
# run this script.

train_X, val_X, train_y, val_y = train_test_split(X, y,test_size=0.3, random_state = 42)
# Define model
model = RandomForestRegressor(random_state=1)
# # fit
model.fit(train_X, train_y)
# predict
preds = model.predict(val_X)
# accuracy
print(mean_absolute_error(val_y, preds))


In [ ]:
print(f"Accuracy: {model.score(val_X, val_y) * 100:.3f}%", )

In [40]:
a = np.array([1,2,3,4])

In [41]:
a

array([1, 2, 3, 4])

In [43]:
df=pd.DataFrame(a)

In [48]:
df=pd.DataFrame()

In [49]:
df

""


In [50]:
df["n"] = pd.DataFrame([11,1,11,1])

In [51]:
df

,n
0,11
1,1
2,11
3,1
